In [1]:
from meteostream import GefsClient
import xarray as xr

In [3]:
client = GefsClient(grib_dir="/home/tylerradebaugh/data/grib")

In [4]:
ds = client.run_GEFS_pipeline(save_file=True, return_ds=True)
ds

Starting GEFS download
Downloaded: /home/tylerradebaugh/data/grib/gefs.wave.t12z.prob.global.0p25.f000.grib2
Downloaded: /home/tylerradebaugh/data/grib/gefs.wave.t12z.prob.global.0p25.f003.grib2
Downloaded: /home/tylerradebaugh/data/grib/gefs.wave.t12z.prob.global.0p25.f006.grib2
Downloaded: /home/tylerradebaugh/data/grib/gefs.wave.t12z.prob.global.0p25.f009.grib2
Downloaded: /home/tylerradebaugh/data/grib/gefs.wave.t12z.prob.global.0p25.f012.grib2
Downloaded: /home/tylerradebaugh/data/grib/gefs.wave.t12z.prob.global.0p25.f015.grib2
Downloaded: /home/tylerradebaugh/data/grib/gefs.wave.t12z.prob.global.0p25.f018.grib2
Downloaded: /home/tylerradebaugh/data/grib/gefs.wave.t12z.prob.global.0p25.f021.grib2
Downloaded: /home/tylerradebaugh/data/grib/gefs.wave.t12z.prob.global.0p25.f024.grib2
Downloaded: /home/tylerradebaugh/data/grib/gefs.wave.t12z.prob.global.0p25.f027.grib2
Downloaded: /home/tylerradebaugh/data/grib/gefs.wave.t12z.prob.global.0p25.f030.grib2
Downloaded: /home/tylerradebaug

: 

In [5]:
import os
# Filter the init index for the var_name
filtered_df = df.loc[client.idx_list]
sel_columns = filtered_df[['var_name', 'grb_shortName']]

# Convert to a dictionary with the idx_ID as keys and (var_name, grb_shortName) as values
var_dict = sel_columns.to_dict('index')
var_dict

{5: {'var_name': 'sigCww4', 'grb_shortName': 'swh'},
 6: {'var_name': 'sigCww5.5', 'grb_shortName': 'swh'},
 7: {'var_name': 'sigCww7', 'grb_shortName': 'swh'},
 8: {'var_name': 'sigCww9', 'grb_shortName': 'swh'},
 38: {'var_name': 'windspd17.48', 'grb_shortName': 'ws'},
 39: {'var_name': 'windspd21.07', 'grb_shortName': 'ws'},
 40: {'var_name': 'windspd24.67', 'grb_shortName': 'ws'}}

In [6]:
datasets = []

for idx, values in var_dict.items():
        var_name, grb_shortName = values['var_name'], values['grb_shortName']

        # Get a list of all GRIB2 files
        grib_files = sorted(os.path.join(client.grib_dir, f) for f in os.listdir(client.grib_dir) if f.startswith(f"GEFS_{var_name}"))
        for file in grib_files:    
            print(file )
            print(f"Processing {grb_shortName} in {file}")
            ds = xr.open_dataset(file, engine='cfgrib', 
                                    backend_kwargs = {"indexpath": None})
            
            # Rename the varibale for xarray to 
            # merge and concat effectively
            ds = ds.rename({grb_shortName:var_name})
            # Drop unnecessary variables and rename dimensions
            print(f"Swapping vars: {grb_shortName} with {var_name}")
            ds = ds.drop_vars('step').rename({'time': 'ref_time'})
            datasets.append(ds)

/home/tylerradebaugh/data/grib/GEFS_sigCww4_gefs_idx5_f000.grib2
Processing swh in /home/tylerradebaugh/data/grib/GEFS_sigCww4_gefs_idx5_f000.grib2
Swapping vars: swh with sigCww4
/home/tylerradebaugh/data/grib/GEFS_sigCww4_gefs_idx5_f003.grib2
Processing swh in /home/tylerradebaugh/data/grib/GEFS_sigCww4_gefs_idx5_f003.grib2
Swapping vars: swh with sigCww4
/home/tylerradebaugh/data/grib/GEFS_sigCww4_gefs_idx5_f006.grib2
Processing swh in /home/tylerradebaugh/data/grib/GEFS_sigCww4_gefs_idx5_f006.grib2
Swapping vars: swh with sigCww4
/home/tylerradebaugh/data/grib/GEFS_sigCww4_gefs_idx5_f009.grib2
Processing swh in /home/tylerradebaugh/data/grib/GEFS_sigCww4_gefs_idx5_f009.grib2
Swapping vars: swh with sigCww4
/home/tylerradebaugh/data/grib/GEFS_sigCww4_gefs_idx5_f012.grib2
Processing swh in /home/tylerradebaugh/data/grib/GEFS_sigCww4_gefs_idx5_f012.grib2
Swapping vars: swh with sigCww4
/home/tylerradebaugh/data/grib/GEFS_sigCww4_gefs_idx5_f015.grib2
Processing swh in /home/tylerradeba

In [ ]:
concat_ds = xr.concat(datasets, dim='valid_time')
concat_ds = concat_ds.fillna(0)

concat_ds

In [17]:
max_value = concat_ds['windspd24.67'].min()

In [18]:
max_value.values

array(0., dtype=float32)

In [22]:
print(type(concat_ds['sigCww4'].values))  # Should print numpy.ndarray

<class 'numpy.ndarray'>
